# Filesystem spec (FSSpec)
FSSpec is the standard package for reading bytes from various stores in the Python/data ecosystem. Intake uses FSSpec as part of their backend.

You can store data at so many places. Local, google drive, Amazon AWS etc. Filesystem Spec (fsspec) is a project to provide a unified pythonic interface to local, remote and embedded file systems and bytes storage. So, all we need is to provide credentials for each service.

For usage purpose, their documentation is more than enough.

Documentation: [link](https://filesystem-spec.readthedocs.io/en/latest/?badge=latest)

## Basic examples
### Local file system without database

In [2]:
import fsspec

# Write a file
with fsspec.open("file://data/fsspec_sample.txt", mode = 'w') as f:
    f.write("Hello, World!\n")

# Read the file
with fsspec.open("file://data/fsspec_sample.txt", mode = 'r') as f:
    print(f.read())

Hello, World!



## Remote filesystem
We will use the S3 filesystem which is operated by MinIO on our local PC.

In [5]:
# Connect to S3 using fsspec

fs = fsspec.filesystem("s3",
                       key = 'minioadmin',
                       secret = 'minioadmin',
                       client_kwargs = {'endpoint_url': 'http://localhost:9000'}
                       )

fs.ls('my-bucket-2')

['my-bucket-2/data', 'my-bucket-2/images']

In [7]:
# Write a file to S3
# Notice the method changed from fsspec.open() to fs.open() when fs is an object that connects to the S3 filesystem.
with fs.open("my-bucket-2/data/fsspec_sample.txt", mode = 'w') as f:
    f.write("Hello, World!")

In [9]:
# Read a file from S3
with fs.open("my-bucket-2/data/data.csv", mode = 'r') as f:
    print(f.read())


with fs.open("my-bucket-2/data/fsspec_sample.txt", mode = 'r') as f:
    print("\n\n"+ f.read())

Name,Age,City
Alice,25,New York
Bob,30,London
Charlie,35,Paris



Hello, World!


## Easily switch between local and remote filesystem

In [12]:
# Read the file
print("Read local file")
with fsspec.open("file://data/fsspec_sample.txt", mode = 'r') as f:
    print(f.read())

print("Read remote file")
with fsspec.open("s3://my-bucket-2/data/fsspec_sample.txt", mode ='r',
                 key = 'minioadmin',
                       secret = 'minioadmin',
                       client_kwargs = {'endpoint_url': 'http://localhost:9000'}) as f:
    print(f.read())

Read local file
Hello, World!

Read remote file
Hello, World!


## Open zarr file with xarray using fsspec

In [15]:
import xarray as xr

store = fsspec.get_mapper("s3://my-bucket-2/data/zarr_data",
                    key = 'minioadmin',
                    secret = 'minioadmin',
                    client_kwargs = {'endpoint_url': 'http://localhost:9000'})

In [16]:
dataset = xr.open_zarr(store)
dataset

KeyError: 'Zarr object is missing the attribute `_ARRAY_DIMENSIONS` and the NCZarr metadata, which are required for xarray to determine variable dimensions.'

I just realised the zarr_data was created using zarr library so xarray can't open it as the metadata is not compatible. Anyways, this the code to open zarr file saved as xarray using fsspec.